In [1]:
import pickle
import pandas as pd
import numpy as np

In [2]:
with open("../merged_data.txt", "rb") as myfile:
    df = pickle.load(myfile)

In [3]:
# get month from released date
def get_month_from_date(date):
    if date == '':
        return float('NaN')
    else:
        return date.split('-')[1]
df['month'] = df['release_date'].apply(get_month_from_date)

In [4]:
df['USAR'] = df['certificates'].apply(lambda x: 1 if type(x) is list and 
                                      ('USA:R' in x) and 
                                      ('Australia:M' in x or 'UK:15' in x or 'UK:18' in x) 
                                      else 0)
df['USAX'] = df['certificates'].apply(lambda x: 1 if type(x) is list and 
                                      'USA:X' in x else 0)
df['USA_production'] = df['countries'].apply(lambda x: 1 if type(x) is list and 
                                             'USA' in x else 0)
df['UK_production'] = df['countries'].apply(lambda x: 1 if type(x) is list and 
                                            'UK' in x else 0)
df['Japan_production'] = df['countries'].apply(lambda x: 1 if type(x) is list and 
                                               'Japan' in x else 0)
df['WG_production'] = df['countries'].apply(lambda x: 1 if type(x) is list and 
                                            'West Germany' in x else 0)
df['HK_production'] = df['countries'].apply(lambda x: 1 if type(x) is list and 
                                            'Hong Kong' in x else 0)
df['India_production'] = df['countries'].apply(lambda x: 1 if type(x) is list and 
                                               'India' in x else 0)
df['pre1930'] = df['year'].apply(lambda x: x < 1930)
df['post2000'] = df['year'].apply(lambda x: x > 2000)
df['revBelow30M'] = df['revenue'].apply(lambda x: (x < 30000000) and (x > 0))
df['revAbove500M'] = df['revenue'].apply(lambda x: x > 500000000)

In [5]:
# drop some columns
df.drop('release_date', axis = 1, inplace = True)
df.drop('certificates', axis = 1, inplace = True)
df.drop('country codes', axis = 1, inplace = True)
df.drop('countries', axis = 1, inplace = True)
df.drop('status', axis = 1, inplace = True)
df.drop('original_language', axis = 1, inplace = True)
df.drop('genres_y', axis = 1, inplace = True)

In [6]:
df.loc[:, 'merge_genres'] = df['merge_genres'].apply(lambda glist:
                                                     'Science Fiction' if glist == 'Sci-Fi'
                                                     else glist)

In [7]:
# convert to numeric data
df['year'] = df['year'].apply(lambda x: float('NaN') if pd.isnull(x) else int(x))
df['votes'] = df['votes'].apply(lambda x: float('NaN') if pd.isnull(x) else int(x))
df['bottom 100 rank'] = df['bottom 100 rank'].apply(lambda x: float('NaN') if pd.isnull(x) else int(x))
df['episode'] = df['episode'].apply(lambda x: float('NaN') if pd.isnull(x) else int(x))

In [8]:
# clean up column "color info"
def parse_color_info(color_info):
    if not (type(color_info) is list):
        return float('NaN')
    elif 'Black and White' in color_info and len(color_info) == 1:
        return 'BW'
    elif 'Black and White' in color_info and len(color_info) > 1:
        return 'BWC'
    else:
        return 'C'
df['color info'] = df['color info'].apply(parse_color_info)

In [9]:
# fill in missing value with mode or mean
for col in df.columns:
    # fill Na
    if df[col].dtypes == object:
        if any(df[col] == ''):
            df.loc[df[col] == '', col] = float('NaN')
        count_null = df[col].isnull().sum()
        if count_null > 0:
            df.loc[:, col] = df[col].fillna(df[col].mode()[0])
    elif col == 'bottom 100 rank':
        df.loc[:, col] = df[col].fillna(0)
    elif col == 'month':
        df.loc[:, col] = df[col].fillna(df[col].mode()[0])
    else: 
        df.loc[:, col] = df[col].fillna(df[col].mean())

In [10]:
from sklearn.preprocessing import MultiLabelBinarizer
y = df['merge_genres'].values
mb = MultiLabelBinarizer()
r = mb.fit_transform(y)
for i,c in enumerate(mb.classes_):
    df[str(c)] = r[:,i]
labels = df[mb.classes_]
genre_bin = []
for i in range(df.shape[0]): 
    temp = ''.join(map(str,labels.iloc[i,:].values))
    genre_bin.append(temp)
df['genre_bin'] = genre_bin

In [11]:
def combine_sf(x):
    if x['Sci-Fi'] == 1 or x['Science Fiction'] == 1:
        return 1
    else:
        return 0
    
df['Science Fiction'] = df.apply(combine_sf, axis = 1)

df.drop('Sci-Fi', axis = 1,inplace=True)

In [12]:
genre_list = ['Action', 'Adult', 'Adventure', 'Animation', 'Biography',
       'Comedy', 'Crime', 'Documentary', 'Drama', 'Family', 'Fantasy',
       'Film-Noir', 'Foreign', 'Game-Show', 'History', 'Horror', 'Music',
       'Musical', 'Mystery', 'News', 'Reality-TV', 'Romance',
       'Science Fiction', 'Short', 'Sport', 'TV Movie', 'Talk-Show',
       'Thriller', 'War', 'Western']

labels = df[genre_list]

genre_bin = []
for i in range(df.shape[0]):
    
    temp = ''.join(map(str,labels.iloc[i,:].values))
    genre_bin.append(temp)
    
df['genre_bin'] = genre_bin

In [13]:
def merge_genres2(x):
    list_b = np.reshape(list(x),(1,-1))[0]
    
    ind_1 = np.where(list_b == '1')[0]
   
    find = [genre_list[i] for i in ind_1]
    return ','.join(find)

# df['genre_bin'][:20].apply(merge_genres2)
df['merge_genres'] = df['genre_bin'].apply(merge_genres2)

In [14]:
with open("../merged_data_v3.pkl", "wb") as myfile:
    pickle.dump(df, myfile)

In [15]:
df.to_csv("../merged_data_v3.csv", index = False)